In [5]:


# import pandas as pd
# import os 
# # from nltk.tokenize import wordpunct_tokenize
# # import estnltk
# from estnltk.vabamorf.morf import syllabify_word
# import string




# file_dir = "/Users/sarah/qp_final/55.txt"

    

# def phrase_to_syll(file_dur):
#     song = open(file_dir).read()
#     phrase_array = song
#     word_array = []
#     words = []
#     frame = pd.DataFrame()
#     for item in phrase_array:
#         tmp = item.split(' ')
#         print(tmp)
#         for word in tmp: 
#             tmpy = word.strip(string.punctuation)
#             tmpz = tmpy.strip('\n')
#             mine = syllabify_word(tmpz,as_dict = True)
#             words.append(mine)
#             syll_col = []
#             q_col = []
#             s_col = []   
#             sec_syll = []
#             sec_q = []
#             sec_s = [] 
#             if len(mine) < 1: 
#                 word_array.append(word)
#                 syll_col.append('NaN')
#                 q_col.append('NaN')
#                 s_col.append('NaN')
#             elif len(mine) == 1:
#                 word_array.append(word)
#                 first = mine[0]
#                 syll_col.append(first.get('syllable'))
#                 q_col.append(first.get('quantity'))
#             elif len(mine) > 1:
#                 word_array.append(word)
#                 first, second, *third  = mine[0:]
#                 # print(first, second, third)
#                 syll_col.append(first.get('syllable'))
#                 q_col.append(first.get('quantity'))
#                 s_col.append(first.get('accent'))
#                 if len(second) > 0: 
#                     sec_syll.append(second.get('syllable'))
#                     sec_q.append(second.get('quantity'))
#                     sec_s.append(second.get('accent'))
#                 else: 
#                     sec_syll.append('NaN')
#                     sec_q.append('NaN')
#                     sec_s.append('NaN')

#     frame['first_syll'] = syll_col
#     frame['first_q']    = q_col
#     frame['first_stress'] = s_col
#     frame['second_syll'] = sec_syll
#     frame['second_quantity'] = sec_q
#     frame['second_stress'] = sec_s
#     return frame


        
# # newer_df
# sunday_df = pd.DataFrame(phrase_to_syll(file_dir))
# sunday_df


In [6]:
import tgt
from os.path import join
import pandas as pd
gridDir = "/Users/sarah/qp_final/txtgridtest/018.TextGrid"

# for fn in os.listdir(gridDir):
# tmp = tgt.read_textgrid(gridDir)
# test = tgt.TextGrid(tmp)
tg_df = pd.DataFrame()
# tiers = tmp.get_tier_names()

def get_duration_label(textgrid, tiername):
    tmp = tgt.read_textgrid(textgrid)
    tg_df = pd.DataFrame()
    mytier = tmp.get_tier_by_name(tiername)
    phrase_dur = mytier.intervals
    dur = []
    label = []
    for interval in phrase_dur:
        dur.append(interval.end_time-interval.start_time)
        label.append(interval.text)
    tg_df["label"] = label 
    tg_df["duration"] = dur
    

    return tg_df
#get tier labels and durations!

# tg_df["phrase"] = (phrase_dur[1]-phrase_dur[0])
# 

    

word_df = get_duration_label(gridDir,"word")



In [7]:
def get_duration_label_interval(textgrid, tiername):
    tmp = tgt.read_textgrid(textgrid)
    tg_df = pd.DataFrame()
    mytier = tmp.get_tier_by_name(tiername)
    phrase_dur = mytier.intervals
    dur = []
    label = []
    for interval in phrase_dur:
        
        dur.append(interval.end_time-interval.start_time)
        label.append(interval.text)
    tg_df["label"] = label 
    tg_df["duration"] = dur
    

    return tg_df

In [8]:
import parselmouth
from estnltk.vabamorf.morf import syllabify_word
import tgt
import string

gridDir2 = "/Users/sarah/qp_final/txtgridtest/69.TextGrid"
def get_duration_labels(textgrid, tiername1,tiername2,tiername3):
    tmp = tgt.read_textgrid(textgrid)
    mytier = tmp.get_tier_by_name(tiername1)
    other = tmp.get_tier_by_name(tiername2)
    ictus = tmp.get_tier_by_name(tiername3)
    

    segments = []

   
    word_dur = mytier.intervals
    
    for interval in word_dur:
        h = interval.start_time
        t = interval.end_time
        b = t-h 
        l = interval.text
        tmpseg = [other.get_annotations_between_timepoints(h,t)]
        s = syllabify_word(l,as_dict=True)
        i = 0 
        for list in tmpseg:
            n = 0 
            while i < len(s):
                item = s[i]
                shape = item.get('syllable')
                q = item.get('quantity')
                a = item.get('accent')
                geminate = False
                for char in shape.strip(string.punctuation):
                    if geminate: continue
                    if n < len(list): 
                        myinterval = list[n]
                        c = myinterval.text
                        d = myinterval.start_time
                        g = myinterval.end_time
                        tmpick = ictus.get_annotations_between_timepoints(d,g,left_overlap=True,right_overlap=True)
                        if len(tmpick) == 0: ick = "off"
                        else: ick = "ictus"
                        j = g-d  
                        mid = g - (j/2)
                        if "ː" in c: 
                            row = [l,b,shape,q,a,c,j,mid,ick]
                            segments.append(row)
                            geminate = True
                            
                        elif char == c: 
                        
                            row = [l,b,shape,q,a,c,j,mid,ick]
                            segments.append(row)
                        else :
                            can = "(" + char + ") " + c
                            row = [l,b,shape,q,a,can,j,mid,ick]
                            segments.append(row)
                        n+=1               
                i+= 1 

            
           


    nu_df = pd.DataFrame(segments,columns=["word","word_dur","syll","quantity","stress","segment","seg_duration","seg_midpoint","ictus"])
    return nu_df

syl_dur_df = get_duration_labels(gridDir2,"word","word/phon","ictus")
syl_dur_df



,word,word_dur,syll,quantity,stress,segment,seg_duration,seg_midpoint,ictus
0,Oh,0.240882,oh,3,1,o,0.165043,0.082521,ictus
1,Oh,0.240882,oh,3,1,h,0.075839,0.202962,ictus
2,me,0.169500,me,3,1,m,0.059664,0.270713,ictus
3,me,0.169500,me,3,1,e,0.109836,0.355463,ictus
4,vaesed,0.619500,vae,2,1,v,0.101258,0.461010,ictus
5,vaesed,0.619500,vae,2,1,a,0.089155,0.556216,ictus
6,vaesed,0.619500,vae,2,1,e,0.153845,0.677716,ictus
7,vaesed,0.619500,sed,2,0,s,0.087913,0.798596,ictus
8,vaesed,0.619500,sed,2,0,e,0.109087,0.897096,off
9,vaesed,0.619500,sed,2,0,d,0.078242,0.990760,off


In [19]:
#get formants
import parselmouth
#make a function! 
test = "/Users/sarah/qp_final/wavs/069.wav"

def get_formants(syl_dur_df, wave):
    song = parselmouth.Sound(wave)
    formant = song.to_formant_burg()
    f1 = []
    f2 = []
    f3 = []
    for float in syl_dur_df.seg_midpoint:
        time = float
        f1.append(formant.get_value_at_time(1,time))
        f2.append(formant.get_value_at_time(2, time))
        f3.append(formant.get_value_at_time(3,time))
    syl_dur_df["f1"] = f1
    syl_dur_df["f2"] = f2
    syl_dur_df["f3"] = f3
    return syl_dur_df
nu_df = get_formants(syl_dur_df,test)
nu_df


,word,word_dur,syll,quantity,stress,segment,seg_duration,seg_midpoint,ictus,f1,f2,f3
0,Oh,0.240882,oh,3,1,o,0.165043,0.082521,ictus,540.905421,989.905678,1414.238432
1,Oh,0.240882,oh,3,1,h,0.075839,0.202962,ictus,850.865128,1620.784985,2186.523346
2,me,0.169500,me,3,1,m,0.059664,0.270713,ictus,346.158399,1818.412065,2798.007639
3,me,0.169500,me,3,1,e,0.109836,0.355463,ictus,411.766557,1385.397329,2214.984918
4,vaesed,0.619500,vae,2,1,v,0.101258,0.461010,ictus,488.221699,1013.678738,1576.019602
5,vaesed,0.619500,vae,2,1,a,0.089155,0.556216,ictus,722.927759,1894.353830,2902.934012
6,vaesed,0.619500,vae,2,1,e,0.153845,0.677716,ictus,701.237833,2083.414680,2469.304151
7,vaesed,0.619500,sed,2,0,s,0.087913,0.798596,ictus,539.898836,2198.275010,3124.359967
8,vaesed,0.619500,sed,2,0,e,0.109087,0.897096,off,492.471769,1896.844298,2225.736269
9,vaesed,0.619500,sed,2,0,d,0.078242,0.990760,off,612.545139,1198.392928,2533.015849


In [26]:
import os
outpath = "/Users/sarah/qp_final/txtgridtest/out"


test = "/Users/sarah/qp_final/txtgridtest/"

songs = "/Users/sarah/qp_final/songs/"

for fn in os.listdir(test):
    if '.TextGrid' not in fn: 
        continue 
    n = fn.strip('.TextGrid') 
    wave = join(songs, n + '.wav')
    data_file = open("ictus_forms_" + n +".csv",'w')
    tmp = pd.DataFrame(get_duration_labels(join(test,fn), "word","word/phon","ictus"))
    nu_df = get_formants(tmp,wave)
    
    print(nu_df)
    nu_df.to_csv(data_file)
    data_file.close()
    
    


    

          word  word_dur syll  quantity  stress segment  seg_duration  \
0        Lõpe,  0.877833   lõ         2       1       l      0.102860   
1        Lõpe,  0.877833   lõ         2       1   (õ) ɵ      0.293082   
2        Lõpe,  0.877833  pe,         2       0       p      0.191140   
3        Lõpe,  0.877833  pe,         2       0       e      0.290751   
4        lõpe,  0.836977   lõ         2       1       l      0.075601   
..         ...       ...  ...       ...     ...     ...           ...   
162  oodakuta!  1.536282    a         2       1   (a) d      0.087159   
163  oodakuta!  1.536282   ku         2       0   (k) a      0.235330   
164  oodakuta!  1.536282   ku         2       0   (u) k      0.200466   
165  oodakuta!  1.536282  ta!         2       0   (t) u      0.276730   
166  oodakuta!  1.536282  ta!         2       0   (a) t      0.185213   

     seg_midpoint  ictus           f1           f2           f3  
0        0.206697  ictus   335.422530  1063.568157  1890.